In [1]:
from dotenv import load_dotenv
import os
load_dotenv() 

False

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [3]:
from datetime import datetime
import pandas as pd
from src.data_utils import load_and_process_taxi_data

from_year = 2024
# to_year = datetime.now().year
to_year = 2025
print(f"Download raw data from {from_year} to {to_year}")

rides = pd.DataFrame()
chunks = []
for year in range(from_year, to_year+1):

    rides_one_year = load_and_process_taxi_data(year)

    chunks.append(rides_one_year)
    break

# Concatenate all chunks at the end
rides = pd.concat(chunks, ignore_index=True)
print("Data loading complete.")

Download raw data from 2024 to 2025
File already exists for 2024-01.
Loading data for 2024-01...
Total records: 1,881,640
Valid records: 57,196
Records dropped: 1,824,444 (96.96%)
Successfully processed data for 2024-01.
File already exists for 2024-02.
Loading data for 2024-02...
Total records: 2,115,518
Valid records: 68,438
Records dropped: 2,047,080 (96.76%)
Successfully processed data for 2024-02.
File already exists for 2024-03.
Loading data for 2024-03...
Total records: 2,655,667
Valid records: 80,317
Records dropped: 2,575,350 (96.98%)
Successfully processed data for 2024-03.
File already exists for 2024-04.
Loading data for 2024-04...
Total records: 3,208,493
Valid records: 98,794
Records dropped: 3,109,699 (96.92%)
Successfully processed data for 2024-04.
File already exists for 2024-05.
Loading data for 2024-05...
Total records: 4,219,327
Valid records: 124,424
Records dropped: 4,094,903 (97.05%)
Successfully processed data for 2024-05.
File already exists for 2024-06.
Loadi

In [4]:
rides

,started_at,pickup_location_id
0,2024-01-22 12:18:53.512,5779
1,2024-01-13 18:52:26.496,5779
2,2024-01-31 16:59:37.612,5779
3,2024-01-04 07:17:28.970,5779
4,2024-01-12 09:06:28.532,5779
...,...,...
1348712,2024-12-31 15:13:28.576,5905
1348713,2024-12-31 10:34:32.031,5905
1348714,2024-12-16 19:57:45.426,5905
1348715,2024-12-24 12:14:00.493,5905


In [5]:
rides.shape

(1348717, 2)

In [6]:
from src.data_utils import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

In [7]:
ts_data.shape

(7325, 3)

In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7325 entries, 0 to 7324
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   pickup_hour         7325 non-null   datetime64[ns]
 1   pickup_location_id  7325 non-null   int16         
 2   rides               7325 non-null   int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 86.0 KB


In [9]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2023-12-31 18:00:00,5626,4
1,2024-01-01 00:00:00,5626,49
2,2024-01-01 06:00:00,5626,42
3,2024-01-01 12:00:00,5626,114
4,2024-01-01 18:00:00,5626,57
...,...,...,...
7320,2024-12-30 18:00:00,6072,80
7321,2024-12-31 00:00:00,6072,15
7322,2024-12-31 06:00:00,6072,87
7323,2024-12-31 12:00:00,6072,142


In [12]:
import hopsworks
import pandas as pd
from hsfs.feature_group import FeatureGroup

project = hopsworks.login()
feature_store = project.get_feature_store()
# Create the feature group
feature_group = feature_store.create_feature_group(
    name="time_series_6h_feature_group",
    version=1,
    description="6 hourly ride data",
    primary_key=[ "pickup_locatin_id", "pickup_hour"],  # Adjust primary key according to your dataset
    event_time="pickup_hour",  # Adjust event time according to your dataset
    online_enabled=True
)
print("Feature group created successfully!")

2025-05-11 05:14:52,112 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 05:14:52,128 INFO: Initializing external client
2025-05-11 05:14:52,128 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-11 05:14:52,952 INFO: Python Engine initialized.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214697
Feature group created successfully!


In [ ]:
# feature_group = feature_store.create_feature_group(
#     name="time_series_hourly_feature_group",
#     version=1,
#     description="Feature group of hourly time series data",  # Use the new 'id' column as the primary key
#     online_enabled=True   # Enable online serving if needed
# )
# print("Feature group created with supported primary key for online serving.")

Feature group created with supported primary key for online serving.


In [13]:
ts_data['pickup_hour'] = pd.to_datetime(ts_data['pickup_hour'])
ts_data['pickup_location_id'] = ts_data['pickup_location_id'].astype(int)
ts_data['rides'] = ts_data['rides'].astype(int)

In [14]:
FEATURE_GROUP_NAME = "time_series_6h_feature_group"
FEATURE_GROUP_VERSION = 1

feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time series data at hourly frequency",
    primary_key=['pickup_location_id', 'pickup_hour'],
    event_time=['pickup_hour']
)

In [15]:
ts_data.rename(columns={
    "pickup_hour": 'pickup_hour',
    "pickup_location_id": 'pickup_location_id',
    "rides": 'rides'
}, inplace=True)

ts_data_selected = ts_data[['pickup_hour', 'pickup_location_id', 'rides']]

feature_group.insert(ts_data_selected, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1214697/fs/1202332/fg/1458597


Uploading Dataframe: 100.00% |██████████| Rows 7325/7325 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: time_series_6h_feature_group_1_offline_fg_materialization


Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214697/jobs/named/time_series_6h_feature_group_1_offline_fg_materialization/executions


(Job('time_series_6h_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)

In [16]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2023-12-31 18:00:00,5626,4
1,2024-01-01 00:00:00,5626,49
2,2024-01-01 06:00:00,5626,42
3,2024-01-01 12:00:00,5626,114
4,2024-01-01 18:00:00,5626,57
...,...,...,...
7320,2024-12-30 18:00:00,6072,80
7321,2024-12-31 00:00:00,6072,15
7322,2024-12-31 06:00:00,6072,87
7323,2024-12-31 12:00:00,6072,142


In [18]:
import hopsworks 
project = hopsworks.login()

2025-05-11 05:21:52,015 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 05:21:52,024 INFO: Initializing external client
2025-05-11 05:21:52,026 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-05-11 05:21:52,648 INFO: Python Engine initialized.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214697


In [19]:
feature_store = project.get_feature_store(name='thanoojl_featurestore')

In [20]:
feature_group = fg = feature_store.get_feature_group('time_series_6h_feature_group', version=1)

In [21]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 7325/7325 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: time_series_6h_feature_group_1_offline_fg_materialization


Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214697/jobs/named/time_series_6h_feature_group_1_offline_fg_materialization/executions


(Job('time_series_6h_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)

In [22]:
df_memory_mb = rides.memory_usage(deep=True).sum() / (1024 * 1024)  
print(f"DataFrame size: {df_memory_mb:.2f} MB")

DataFrame size: 30.87 MB


In [23]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7325 entries, 0 to 7324
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   pickup_hour         7325 non-null   datetime64[ns]
 1   pickup_location_id  7325 non-null   int32         
 2   rides               7325 non-null   int32         
dtypes: datetime64[ns](1), int32(2)
memory usage: 114.6 KB
